# DSE 241 - Final Project
# Rock & Fucking Roll

## Requirements

In [27]:
import requests
import pandas as pd
import numpy as np

## Functions

In [28]:
def read_api_key(file_name):
    f = open(file_name, 'r')
    key = f.read()
    f.close()
    return key

def getgenre_artist(string):
    ## lower case
    ## replace " " with "_"
    artist = string.replace(' ','_').lower()
    return artist

def bands_artist(string):
    ## url-ify artist names
    ## replaces " " with "%20"
    artist = string.replace(' ',r'%20')
    return artist
    
    
def parse_events(event_json):
    ### defined to help parse the JSON of individual events returned by Bandintown API ###
    ### this will pull out the keys and values for the event object ###
    ### the end result is a list that can be entered as a row to a Pandas dataframe ###
    
    ### TODO: determine if the output is better in Pandas or Numpy ###
    ### TODO: define event_json object ###
    
    events_data = []
    num_of_events = len(event_json) # event json has an entry for each event ## each event has lots of other nested data

    keys_of_interest = [
            'datetime' # date-time of the event
            , 'title'  # name of the event  ## may need to create a condition to check if exists and set to null if missing
            , 'lineup' # list of strings containing names of artists  ## may leave as a nested list in data frame to avoid sparseness  ### list may be ordered in terms of headliners (??)
            , 'festival_start_date' # date festival starts  ### may be a useful indicator that the artist is performing at a festival-event, need to test this for reliability
            , 'festival_end_date'   # date festival ends, will differ from start date on multi-day events (may not exist on single day events)
            , 'venue'  # this is a nested dictionary, will need to tease this one out to flatten the data
    ]

    venue_keys_of_interest = [
             'city'        # city name, string
            , 'region'     # state-level, string
            , 'country'    # country name, string
            , 'latitude'   # coordinate data, float
            , 'longitude'  # coordinate data, float
            , 'location'   # arbitrary string describing geolocation  ## consider not including
            , 'name'       # arbitrary string describing venue name, could be misleading since some venues are at locations, but given festival name  ## consider not including
    ]

    for i in range(num_of_events):
        event = event_json[i]
        event_list = []

        for key in keys_of_interest:
            value = event.get(key)
            if key == 'venue':
                for venue_key in venue_keys_of_interest:
                    venue_value = value.get(venue_key)
                    event_list.append(venue_value)
            else:
                event_list.append(value)

        events_data.append(event_list)


    cols = keys_of_interest[:-1] + venue_keys_of_interest
    events_df = pd.DataFrame(events_data, columns=cols)
    events_df['artist'] = artist
    return events_df

## Key Inputs

In [29]:
artist = 'Tame Impala'

bands_key = read_api_key('bands_api_key.txt') ## shhhh... its a secret

analysis_level = 1
gg_artist = getgenre_artist(artist)
bands_artist = bands_artist(artist)
getgenre_api_url = r'https://api.getgenre.com/search?artist_name={}&analysis={}'.format(gg_artist, analysis_level)
bands_api_url = r'https://rest.bandsintown.com/artists/{}/events/?app_id={}'.format(bands_artist, bands_key)

In [30]:
gg_artist

'tame_impala'

In [31]:
bands_artist

'Tame%20Impala'

## Working Code
### Let's Go

In [32]:
r = requests.get(bands_api_url)

In [33]:
event_json = r.json()

In [34]:
test_df = parse_events(event_json)
print(test_df.shape)
test_df.head()

(24, 13)


,datetime,title,lineup,festival_start_date,festival_end_date,city,region,country,latitude,longitude,location,name,artist
0,2022-02-26T19:00:00,Innings Festival 2022,"[Tame Impala, Foo Fighters, My Morning Jacket,...",2022-02-26,2022-02-27,Tempe,AZ,United States,33.4307373,-111.94169360000001,"Tempe, AZ",Innings Festival 2022,Tame Impala
1,2022-03-03T19:00:00,,"[Tame Impala, Pepper, Flying Lotus, Caribou, D...",,,Okeechobee,FL,United States,27.35988,-80.73357399999998,"Okeechobee, FL",Sunshine Grove - FL,Tame Impala
2,2022-03-07T20:00:00,,[Tame Impala],,,Pittsburgh,PA,United States,40.4438281,-79.96228339999999,"Pittsburgh, PA",Petersen Events Center,Tame Impala
3,2022-03-09T20:00:00,,"[Tame Impala, Sudan Archives]",,,Toronto,ON,Canada,43.643466,-79.379142,"Toronto, ON",Scotiabank Arena,Tame Impala
4,2022-03-10T20:00:00,,[Tame Impala],,,Laval,QC,Canada,45.5558363,-73.7214777,"Laval, QC",Place Bell,Tame Impala


In [35]:
r_gg = requests.get(getgenre_api_url)

In [60]:
genre_json = r_gg.json()
top_gg = genre_json['analysis']['top_genres']

In [61]:
test_df['artist'] = artist
test_df['artist_topgenres'] = [top_gg for _ in range(len(test_df))]
test_df.head()

,datetime,title,lineup,festival_start_date,festival_end_date,city,region,country,latitude,longitude,location,name,artist,artist_topgenres
0,2022-02-26T19:00:00,Innings Festival 2022,"[Tame Impala, Foo Fighters, My Morning Jacket,...",2022-02-26,2022-02-27,Tempe,AZ,United States,33.4307373,-111.94169360000001,"Tempe, AZ",Innings Festival 2022,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p..."
1,2022-03-03T19:00:00,,"[Tame Impala, Pepper, Flying Lotus, Caribou, D...",,,Okeechobee,FL,United States,27.35988,-80.73357399999998,"Okeechobee, FL",Sunshine Grove - FL,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p..."
2,2022-03-07T20:00:00,,[Tame Impala],,,Pittsburgh,PA,United States,40.4438281,-79.96228339999999,"Pittsburgh, PA",Petersen Events Center,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p..."
3,2022-03-09T20:00:00,,"[Tame Impala, Sudan Archives]",,,Toronto,ON,Canada,43.643466,-79.379142,"Toronto, ON",Scotiabank Arena,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p..."
4,2022-03-10T20:00:00,,[Tame Impala],,,Laval,QC,Canada,45.5558363,-73.7214777,"Laval, QC",Place Bell,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p..."


In [62]:
test_df['location'].value_counts()

Amsterdam, Netherlands    3
Asheville, NC             2
Brooklyn, NY              2
Nashville, TN             1
Auckland, New Zealand     1
Milan, Italy              1
London, United Kingdom    1
Copenhagen, Denmark       1
Louisville, KY            1
New Orleans, LA           1
Tempe, AZ                 1
Okeechobee, FL            1
Philadelphia, PA          1
Hampton, VA               1
Boston, MA                1
Uncasville, CT            1
Laval, QC                 1
Toronto, ON               1
Pittsburgh, PA            1
México, Mexico            1
Name: location, dtype: int64

In [64]:
test_df['lineup'].str.len().value_counts()

1     17
17     1
33     1
2      1
49     1
47     1
25     1
26     1
Name: lineup, dtype: int64

In [70]:
test_df['lineup_size'] = test_df['lineup'].str.len()
test_df['festival_flag'] = np.where(test_df['lineup'].str.len() > 5, 1, 0)
test_df.head()

,datetime,title,lineup,festival_start_date,festival_end_date,city,region,country,latitude,longitude,location,name,artist,artist_topgenres,festival_flag,lineup_size
0,2022-02-26T19:00:00,Innings Festival 2022,"[Tame Impala, Foo Fighters, My Morning Jacket,...",2022-02-26,2022-02-27,Tempe,AZ,United States,33.4307373,-111.94169360000001,"Tempe, AZ",Innings Festival 2022,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p...",1,17
1,2022-03-03T19:00:00,,"[Tame Impala, Pepper, Flying Lotus, Caribou, D...",,,Okeechobee,FL,United States,27.35988,-80.73357399999998,"Okeechobee, FL",Sunshine Grove - FL,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p...",1,33
2,2022-03-07T20:00:00,,[Tame Impala],,,Pittsburgh,PA,United States,40.4438281,-79.96228339999999,"Pittsburgh, PA",Petersen Events Center,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p...",0,1
3,2022-03-09T20:00:00,,"[Tame Impala, Sudan Archives]",,,Toronto,ON,Canada,43.643466,-79.379142,"Toronto, ON",Scotiabank Arena,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p...",0,2
4,2022-03-10T20:00:00,,[Tame Impala],,,Laval,QC,Canada,45.5558363,-73.7214777,"Laval, QC",Place Bell,Tame Impala,"[indie rock, psychedelic rock, synthpop, neo-p...",0,1


## Messy Workspace -- May not Run

In [77]:
import plotly.express as px
#df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(test_df,
                     lat='latitude',
                     lon='longitude',
                     color="festival_flag", # which column to use to set the color of markers
                     hover_name="city", # column added to hover information
                     #size="lineup_size", # size of markers
                     projection="natural earth")
fig.update_geos(
    visible=False,
    resolution=50,
    #scope="north america",
    showcountries=True, countrycolor="Black"
    #,showsubunits=True, subunitcolor="Blue"
)
fig.show()

In [81]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattergeo(
    test_df,
    lat='latitude',
    lon='longitude',
    color="festival_flag", # which column to use to set the color of markers
    hover_name="city", # column added to hover information
    #size="lineup_size", # size of markers
    projection="natural earth")
)
fig.update_geos(
    visible=False, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="lightGrey"
)
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: The first argument to the plotly.graph_objs.Scattergeo 
constructor must be a dict or 
an instance of :class:`plotly.graph_objs.Scattergeo`

In [82]:
import pandas as pd
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")
us_cities.head()

import plotly.express as px

fig = px.scatter_mapbox(us_cities, lat="lat", lon="lon", hover_name="City", hover_data=["State", "Population"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [86]:
import plotly.express as px

fig = px.scatter_mapbox(test_df, lat="latitude", lon="longitude", hover_name="location",
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

TypeError: Could not convert 33.430737327.3598840.443828143.64346645.555836341.491597440.6824952000000140.6824952000000142.36619837.02779539.9012015000000135.5973135.5973136.159173629.935031738.27900355.6252113999999951.536561452.312523352.312523352.312523345.46427-36.8472420000000119.4039371 to numeric

In [37]:
ej = r.json()
ej[2]

{'offers': [{'type': 'Tickets',
   'url': 'https://www.bandsintown.com/t/1023669640?app_id=8a5eb4a34e1366162ba48546a8a63976&came_from=267&utm_medium=api&utm_source=public_api&utm_campaign=ticket',
   'status': 'available'}],
 'venue': {'country': 'United States',
  'city': 'Pittsburgh',
  'latitude': '40.4438281',
  'name': 'Petersen Events Center',
  'location': 'Pittsburgh, PA',
  'region': 'PA',
  'longitude': '-79.96228339999999'},
 'festival_datetime_display_rule': '',
 'description': 'Tame Impala',
 'lineup': ['Tame Impala'],
 'festival_start_date': '',
 'bandsintown_plus': False,
 'title': '',
 'artist_id': '208845',
 'url': 'https://www.bandsintown.com/e/1023669640?app_id=8a5eb4a34e1366162ba48546a8a63976&came_from=267&utm_medium=api&utm_source=public_api&utm_campaign=event',
 'datetime': '2022-03-07T20:00:00',
 'on_sale_datetime': '2021-12-10T10:00:00',
 'id': '1023669640',
 'festival_end_date': ''}

In [38]:
len(event_json)

24

In [39]:
#r.text

In [40]:
len(ej[0])

15

In [41]:
ej[0].keys()

dict_keys(['offers', 'venue', 'artist', 'festival_datetime_display_rule', 'description', 'lineup', 'festival_start_date', 'bandsintown_plus', 'title', 'artist_id', 'url', 'datetime', 'on_sale_datetime', 'id', 'festival_end_date'])

In [42]:
list(ej[0].keys())

['offers',
 'venue',
 'artist',
 'festival_datetime_display_rule',
 'description',
 'lineup',
 'festival_start_date',
 'bandsintown_plus',
 'title',
 'artist_id',
 'url',
 'datetime',
 'on_sale_datetime',
 'id',
 'festival_end_date']

In [43]:
type(ej[0])

dict

In [44]:
ej[0].values()

dict_values([[{'type': 'Tickets', 'url': 'https://www.bandsintown.com/t/1023143559?app_id=8a5eb4a34e1366162ba48546a8a63976&came_from=267&utm_medium=api&utm_source=public_api&utm_campaign=ticket', 'status': 'available'}], {'country': 'United States', 'city': 'Tempe', 'latitude': '33.4307373', 'name': 'Innings Festival 2022', 'location': 'Tempe, AZ', 'region': 'AZ', 'longitude': '-111.94169360000001'}, {'thumb_url': 'https://photos.bandsintown.com/thumb/9092561.jpeg', 'mbid': '63aa26c3-d59b-4da4-84ac-716b54f1ef4d', 'facebook_page_url': 'http://www.facebook.com/35478245776', 'image_url': 'https://photos.bandsintown.com/large/9092561.jpeg', 'tracker_count': 1668071.0, 'tracking': [], 'upcoming_event_count': 24.0, 'url': 'https://www.bandsintown.com/a/208845?came_from=267&app_id=8a5eb4a34e1366162ba48546a8a63976', 'support_url': '', 'name': 'Tame Impala', 'options': {'display_listen_unit': False}, 'links': [{'type': 'website', 'url': 'https://official.tameimpala.com/'}, {'type': 'facebook', 

In [45]:
v = ej[0].get('venue')
print(v)
type(v)

{'country': 'United States', 'city': 'Tempe', 'latitude': '33.4307373', 'name': 'Innings Festival 2022', 'location': 'Tempe, AZ', 'region': 'AZ', 'longitude': '-111.94169360000001'}


dict

In [46]:
v.get('country')

'United States'

In [47]:
for key in list(ej[0].keys()):
    print(key, " : \n", ej[0].get(key), "\n")

offers  : 
 [{'type': 'Tickets', 'url': 'https://www.bandsintown.com/t/1023143559?app_id=8a5eb4a34e1366162ba48546a8a63976&came_from=267&utm_medium=api&utm_source=public_api&utm_campaign=ticket', 'status': 'available'}] 

venue  : 
 {'country': 'United States', 'city': 'Tempe', 'latitude': '33.4307373', 'name': 'Innings Festival 2022', 'location': 'Tempe, AZ', 'region': 'AZ', 'longitude': '-111.94169360000001'} 

artist  : 
 {'thumb_url': 'https://photos.bandsintown.com/thumb/9092561.jpeg', 'mbid': '63aa26c3-d59b-4da4-84ac-716b54f1ef4d', 'facebook_page_url': 'http://www.facebook.com/35478245776', 'image_url': 'https://photos.bandsintown.com/large/9092561.jpeg', 'tracker_count': 1668071.0, 'tracking': [], 'upcoming_event_count': 24.0, 'url': 'https://www.bandsintown.com/a/208845?came_from=267&app_id=8a5eb4a34e1366162ba48546a8a63976', 'support_url': '', 'name': 'Tame Impala', 'options': {'display_listen_unit': False}, 'links': [{'type': 'website', 'url': 'https://official.tameimpala.com/'